<a href="https://colab.research.google.com/github/BsharaArz/fp8/blob/exercise/mlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from jax import numpy as jnp
import jax
from typing_extensions import NamedTuple

In [ ]:
class MLP(NamedTuple):
  # whatever parameters you need - usually the up projection and down projection

  def init_mlp(self, prng_key: jax.Array, d_model: int, d_ff: int):
    self.prng_key = prng_key
    self.d_model = d_model
    self.d_ff = d_ff
    self.dtype = jnp.float32

    #initialize params
    initializer = jax.nn.initializers.normal(0.01) #stan-dev = 0.01
    self.parameters = [[initializer(prng_key, (d_ff,d_model), self.dtype), initializer(prng_key, (d_ff, d_model), self.dtype)], [initializer(prng_key, (d_model, d_ff), self.dtype), initializer(prng_key, (d_model, d_model), self.dtype)]]
    return self
    # initialize the parameters and return an instance of MLP

  def forward_mlp(self, seq: jax.Array):
    params = self.parameters
    # seq is Sequence - input to the MLP block.
    # seq is of shape (batch_size, sequence_length, d_model)
    batch_size, sequence_length, d_model = seq.shape

    # Do the necessary matrix multiplications and return the transformer sequence
    activations = seq.reshape(batch_size * sequence_length, d_model)
    for w, b in params[:-1]:
      activations = jax.nn.relu(jnp.dot(w, activations) + b)

    final_w, final_b = params[-1]
    logits = jnp.dot(final_w, activations) + final_b

    #softmax at end
    final = jax.nn.softmax(logits)
    return final.reshape(batch_size, sequence_length, d_model)



  def test_mlp():
    prng_key = jax.random.PRNGKey(0)
    d_model = 512
    d_ff = 2048

    batch_size = 32
    sequence_length = 16

    initializer = jax.nn.initializers.normal(0.01)
    seq = initializer(prng_key, (batch_size, sequence_length, d_model), jnp.float32)

    print('input shape: ')
    print(seq.shape)

    mlp = MLP.init_mlp(MLP, prng_key, d_model, d_ff)
    output = mlp.forward_mlp(mlp, seq)
    print('output shape: ')
    print(output.shape)

    # Create a random tensor for sequence
    # Create the MLP using init_mlp
    # Pass the sequence through the MLP using forward_mlp and capture the output
    # Assert that the output if of the correct shape

  def main():
    return MLP.test_mlp()

In [ ]:
MLP.main()

input shape: 
(32, 16, 512)
output shape: 
(32, 16, 512)
